### Search for assets and observations in ArcGIS Field Maps

This notebook demonstrates how you can configure search in the layer of a web maps. The search is then honored in apps, including ArcGIS Field Maps.


#### Connect to your ArcGIS Online or ArcGIS Enterprise organization

In [ ]:
import json

from arcgis.gis import GIS
from arcgis.mapping import WebMap

print('Enter your ArcGIS account user name: ')
username = input()
gis = GIS('https://www.arcgis.com', username)
print('Connected to {}'.format(gis.properties.portalHostname))

#### Get the web map

Search by ID for the web map where you want to configure layer search. 

In [ ]:

# update below with web map id
webmap_item_no_search = gis.content.get('<web map item id>')

#### Add search to a feature layer

If the map already has feature search configured, add another search to it. If not, insert the first feature search.

In [ ]:
def add_layer_search(webmap_item, layer_url, field_name, exact_match, esri_field_type):
    json_data = webmap_item.get_data()
    
    # insert placeholder into JSON if not present
    if 'applicationProperties' not in json_data:
        placeholder_app_properties = {'viewing': {'search': {'enabled': True,'disablePlaceFinder': False,'hintText': 'Place or Address','layers': [],'tables': []}}}
        json_data['applicationProperties'] = placeholder_app_properties
  
    app_properties_viewing = json_data['applicationProperties']['viewing']

    # Get the ID of the layer to search, based on the URL you know it has
    webmap = WebMap(webmap_item)
    layers = webmap.layers
    layer_id = None
    for layer in layers:
        if layer.url == layer_url:
            layer_id = layer.id
        
    if layer_id is None:
        print('Layer URL not found: ' + layer_url, end='\n\n')
        return
 
    if ('search' in app_properties_viewing):
        
        search_layers_property = app_properties_viewing['search']['layers']
        new_layer = {'id': layer_id, 'field': {'name': field_name, 'exactMatch': exact_match, 'type': esri_field_type}}
        search_layers_property.append(new_layer)
        webmap_item.update(data = json.dumps(json_data))
        print("Added another search in '" + webmap_item.title + "`", end='\n\n')
        

#### Disable place search

If you won't want mobile workers searching for generic places and addresses, disable place search.

In [ ]:
def disable_place_search(webmap_item):

    json_data = webmap_item.get_data()

    # insert placeholder into JSON if not present
    if 'applicationProperties' not in json_data:
        placeholder_app_properties = {'viewing': {'search': {'enabled': True,'disablePlaceFinder': False,'hintText': 'Place or Address','layers': [],'tables': []}}}
        json_data['applicationProperties'] = placeholder_app_properties

    app_properties_viewing = json_data['applicationProperties']['viewing']

    if ('search' in app_properties_viewing):
    
        layer_search_property = app_properties_viewing['search']
        disable_place_finder = {'disablePlaceFinder': True}
        layer_search_property.update(disable_place_finder)
        webmap_item.update(data = json.dumps(json_data))
        
        print("Disabled place finder for '" + webmap_item.title + "'")

#### Provide hint text

Check if it has layer search configured, and if so, provide hint text.

In [ ]:
def update_hint_text(webmap_item,new_hint_text):

    json_data = webmap_item.get_data()

    # insert placeholder into JSON if not present
    if 'applicationProperties' not in json_data:
        placeholder_app_properties = {'viewing': {'search': {'enabled': True,'disablePlaceFinder': False,'hintText': 'Place or Address','layers': [],'tables': []}}}
        json_data['applicationProperties'] = placeholder_app_properties
    
    app_properties_viewing = json_data['applicationProperties']['viewing']

    if ('search' in app_properties_viewing):
    
        layer_search_property = app_properties_viewing['search']
        hint_text_dict = {'hintText': new_hint_text}
        layer_search_property.update(hint_text_dict)
        webmap_item.update(data = json.dumps(json_data))

        print("Updated hint text for '" + webmap_item.title + "'")   

#### Configure layer and process updates

In [ ]:
webmap_item = webmap_item_no_search

# update below with our layer url and field details
layer_url = 'https://services.arcgis.com/N4jtru9dctSQR53c/arcgis/rest/services/Hydrant_maintenance/FeatureServer/0'

add_layer_search(webmap_item, layer_url, 'Facility Identifier', True, 'esriFieldTypeInteger') #ID
add_layer_search(webmap_item, layer_url, 'Creator', False, 'esriFieldTypeString') #Details

update_hint_text(webmap_item,"Search for hydrant...")
disable_place_search(webmap_item)